In [1]:
!python3 -m pip install --upgrade kaggle

In [2]:
!kaggle

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [3]:
!cp -R /content/kaggle.json /root/.kaggle/

cp: cannot stat '/content/kaggle.json': No such file or directory


In [4]:
!chmod 600 /root/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [5]:

!export KAGGLE_USERNAME=pragnakalp
!export KAGGLE_KEY=aef332e92e7cc55bf0b6b6f1eed2f1cf

In [6]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

Dataset URL: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
License(s): CC0-1.0
100% 228M/228M [00:11<00:00, 30.7MB/s]
100% 228M/228M [00:11<00:00, 20.9MB/s]


In [7]:
!unzip the-movies-dataset.zip

Archive:  the-movies-dataset.zip
  inflating: credits.csv             
  inflating: keywords.csv            
  inflating: links.csv               
  inflating: links_small.csv         
  inflating: movies_metadata.csv     
  inflating: ratings.csv             
  inflating: ratings_small.csv       


In [8]:
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
import pandas as pd

In [10]:
df_meta = pd.read_csv("movies_metadata.csv")

<ipython-input-10-41f2d4b3cb26>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta = pd.read_csv("movies_metadata.csv")


In [11]:
df_meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [12]:
df_meta = df_meta[['id','genres', 'original_language', 'production_countries', 'tagline', 'original_title', 'adult','release_date','status']]

In [13]:
df_meta.status.unique()

array(['Released', nan, 'Rumored', 'Post Production', 'In Production',
       'Planned', 'Canceled'], dtype=object)

In [14]:
for i in range(len(df_meta)):
  tmp = df_meta['status'][i]
  if (tmp == "Released"):
    df_meta['status'][i] = "Released"
  else:
    df_meta['status'][i] = ''

In [15]:
import json
import ast

In [16]:
df_meta['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [17]:
for i in range(len(df_meta['genres'])):
  str_tmp = ""
  genre = df_meta['genres'][i]
  genre = genre.replace("\'", "\"")
  json_genre = json.loads(genre)
  for j in range(len(json_genre)):
    str_tmp += (json_genre[j]['name'])+" "
  df_meta['genres'][i] = str_tmp

In [18]:
df_meta['genres'][0]

'Animation Comedy Family '

In [19]:
df_meta['production_countries'][0]

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [20]:
df_meta['production_countries'].replace(np.nan, '', inplace=True)

In [21]:
for i in range(len(df_meta['production_countries'])):
  str_tmp = ""
  country = df_meta['production_countries'][i]
  if (country != ''):
    country = json.dumps(ast.literal_eval(country))
    json_country = json.loads(country)
    print(json_country)
    try:
      for j in range(len(json_country)):
        str_tmp += (json_country[j]['name'])
      print(str_tmp)

      df_meta['production_countries'][i] = str_tmp
    except:
      print("Error ================> ")
  else:
    print("Blank")

Streaming output truncated to the last 5000 lines.
[{'iso_3166_1': 'IE', 'name': 'Ireland'}]
Ireland
[{'iso_3166_1': 'ES', 'name': 'Spain'}]
Spain
[]

[]

[{'iso_3166_1': 'US', 'name': 'United States of America'}]
United States of America
[{'iso_3166_1': 'RU', 'name': 'Russia'}]
Russia
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
United States of America
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
United States of America
[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]
United Kingdom
[{'iso_3166_1': 'RU', 'name': 'Russia'}, {'iso_3166_1': 'US', 'name': 'United States of America'}, {'iso_3166_1': 'DE', 'name': 'Germany'}]
RussiaUnited States of AmericaGermany
[{'iso_3166_1': 'CA', 'name': 'Canada'}]
Canada
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
United States of America
[{'iso_3166_1': 'DE', 'name': 'Germany'}]
Germany
[{'iso_3166_1': 'FR', 'name': 'France'}]
France
[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}, {'iso_3166_1': 'JP', 'na

In [22]:
df_meta['production_countries'][0]

'United States of America'

In [23]:
df_keyword = pd.read_csv('keywords.csv')

In [24]:
df_keyword['keywords'][0]

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

In [25]:
for i in range(len(df_keyword['keywords'])):
  str_tmp = ""
  keyword = df_keyword['keywords'][i]
  keyword = json.dumps(ast.literal_eval(keyword))
  json_keyword = json.loads(keyword)
  # print(json_keyword)
  for j in range(len(json_keyword)):
    str_tmp += (json_keyword[j]['name'])+" "
  # print(str_tmp)

  df_keyword['keywords'][i] = str_tmp

Streaming output truncated to the last 5000 lines.
<ipython-input-25-3aa342bc5694>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_keyword['keywords'][i] = str_tmp
<ipython-input-25-3aa342bc5694>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_keyword['keywords'][i] = str_tmp
<ipython-input-25-3aa342bc5694>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_keyword['keywords'][i] = str_tmp
<ipython-input-25-3aa342bc5694>:11

In [26]:
df_keyword['keywords'][0]

'jealousy toy boy friendship friends rivalry boy next door new toy toy comes to life '

In [27]:
df_keyword['keywords'][1]

"board game disappearance based on children's book new home recluse giant insect "

In [28]:
df_credit = pd.read_csv('credits.csv')

In [29]:
df_credit = df_credit.rename(columns=({'crew':'director'}))

In [30]:
df_credit['cast'][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [31]:
# for i in range(len(df_credit['cast'])):
#   str_tmp = ""
#   credit = df_credit['cast'][i]
#   credit = json.dumps(ast.literal_eval(credit))
#   json_credit = json.loads(credit)
#   # print(json_credit[0]['name'])

#   for j in range(len(json_credit)):
#     str_tmp += (json_credit[j]['name'])+" "
#   # print(str_tmp)

#   df_credit['cast'][i] = str_tmp
for i in range(len(df_credit['cast'])):
  cast_string = df_credit['cast'][i]
  # Split the string based on a delimiter (e.g., space or comma)
  actor_list = cast_string.split(" ")  # Adjust delimiter if needed
  str_tmp = " ".join(actor_list)  # Join back with spaces
  df_credit['cast'][i] = str_tmp

Streaming output truncated to the last 5000 lines.
<ipython-input-31-5b21f5ac4206>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit['cast'][i] = str_tmp
<ipython-input-31-5b21f5ac4206>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit['cast'][i] = str_tmp
<ipython-input-31-5b21f5ac4206>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit['cast'][i] = str_tmp
<ipython-input-31-5b21f5ac4206>:18: SettingWithCo

In [32]:
df_credit['cast'][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [33]:
director = df_credit['director'][0]

In [34]:
print(director)

[{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name': 'John Lasseter', 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}, {'credit_id': '52fe4284c3a36847f8024f4f', 'department': 'Writing', 'gender': 2, 'id': 12891, 'job': 'Screenplay', 'name': 'Joss Whedon', 'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'}, {'credit_id': '52fe4284c3a36847f8024f55', 'department': 'Writing', 'gender': 2, 'id': 7, 'job': 'Screenplay', 'name': 'Andrew Stanton', 'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'}, {'credit_id': '52fe4284c3a36847f8024f5b', 'department': 'Writing', 'gender': 2, 'id': 12892, 'job': 'Screenplay', 'name': 'Joel Cohen', 'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'}, {'credit_id': '52fe4284c3a36847f8024f61', 'department': 'Writing', 'gender': 0, 'id': 12893, 'job': 'Screenplay', 'name': 'Alec Sokolow', 'profile_path': '/v79vlRYi94BZUQnkkyznbGUZLjT.jpg'}, {'credit_id': '52fe4284c3a36847f8024f67', 'depart

In [35]:
for i in range(len(df_credit['director'])):
  str_tmp = ""
  director = df_credit['director'][i]
  director = json.dumps(ast.literal_eval(director))
  json_director = json.loads(director)
  # print(json_director)
  for j in range(len(json_director)):
    if json_director[j]['job'] == 'Director':
      str_tmp += (json_director[j]['name'])+" "
      # print(str_tmp)
  df_credit['director'][i] = str_tmp

Streaming output truncated to the last 5000 lines.
<ipython-input-35-6e6831e01b48>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit['director'][i] = str_tmp
<ipython-input-35-6e6831e01b48>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit['director'][i] = str_tmp
<ipython-input-35-6e6831e01b48>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit['director'][i] = str_tmp
<ipython-input-35-6e6831e01b48>:11: S

In [36]:
df_credit['director'][0]

'John Lasseter '

In [37]:
df_keyword['id']=df_keyword['id'].astype(str)

In [38]:
df_meta['id']=df_meta['id'].astype(str)

In [39]:
df_merge = pd.merge(df_keyword, df_meta, on='id', how='inner')[['id', 'genres', 'original_language', 'production_countries', 'tagline',
       'original_title', 'keywords', 'adult', 'release_date', 'status']]

In [40]:
df_credit['id']=df_credit['id'].astype(str)

In [41]:
df_merge_whole = pd.merge(df_merge, df_credit, on='id', how='inner')[['id', 'genres', 'original_language', 'production_countries', 'tagline',
       'original_title', 'keywords', 'cast','director', 'adult', 'release_date', 'status']]

In [42]:
df_merge_whole['keywords'].replace('', np.nan, inplace=True)

In [43]:
df_merge_whole['genres'].replace('', np.nan, inplace=True)

In [44]:
df_merge_whole['original_title'].replace('', np.nan, inplace=True)

In [45]:
df_merge_whole['cast'].replace('', np.nan, inplace=True)

In [46]:
df_merge_whole['director'].replace('', np.nan, inplace=True)

In [47]:
df_merge_whole['release_date'].replace('', np.nan, inplace=True)

In [48]:
df_merge_whole['status'].replace('', np.nan, inplace=True)

In [49]:
df_merge_whole['production_countries'].replace(np.nan, '', inplace=True)

In [50]:
df_merge_whole['adult'].replace(np.nan, '', inplace=True)

In [51]:
df_merge_whole['tagline'].replace(np.nan, '', inplace=True)

In [52]:
df_merge_whole = df_merge_whole.dropna()

Save the filtered data to CSV and read the new CSV

In [53]:
df_merge_whole.to_csv('filter_data.csv')

In [54]:
df = pd.read_csv('filter_data.csv')

### **Combine the columns**

Now we will combine the columns on which we’ll later perform embedding.

In [55]:
def combine_features(row):
    return row['original_title']+' '+row['genres']+' '+ row['original_language']
    +' '+row['director']+' '+row['keywords']+' '+row['cast']+' '+row['tagline']
    +' '+row['production_countries']

In [56]:
df['combined_value'] = df.apply(combine_features, axis = 1)

In [57]:
df.head()

,Unnamed: 0,id,genres,original_language,production_countries,tagline,original_title,keywords,cast,director,adult,release_date,status,combined_value
0,0,862,Animation Comedy Family,en,United States of America,NaN,Toy Story,jealousy toy boy friendship friends rivalry bo...,"[{'cast_id': 14, 'character': 'Woody (voice)',...",John Lasseter,False,1995-10-30,Released,Toy Story Animation Comedy Family en
1,1,8844,Adventure Fantasy Family,en,United States of America,Roll the dice and unleash the excitement!,Jumanji,board game disappearance based on children's b...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...",Joe Johnston,False,1995-12-15,Released,Jumanji Adventure Fantasy Family en
2,2,15602,Romance Comedy,en,United States of America,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,fishing best friend duringcreditsstinger old men,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...",Howard Deutch,False,1995-12-22,Released,Grumpier Old Men Romance Comedy en
3,3,31357,Comedy Drama Romance,en,United States of America,Friends are the people who let you be yourself...,Waiting to Exhale,based on novel interracial relationship single...,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...",Forest Whitaker,False,1995-12-22,Released,Waiting to Exhale Comedy Drama Romance en
4,4,11862,Comedy,en,United States of America,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,baby midlife crisis confidence aging daughter ...,"[{'cast_id': 1, 'character': 'George Banks', '...",Charles Shyer,False,1995-02-10,Released,Father of the Bride Part II Comedy en


Let’s also create an index column for movies.

In [58]:
df['index'] = [i for i in range(0, len(df))]

### **Create Functions**

Create two functions as given below:

1. title: To get the title of the movie from index.

2. index: To get the index of the movie from title.

In [59]:
# Functions

#Get title of movie
def title(index):
    return df[df.index == index]["original_title"].values[0]

#Get index of movie
def index(original_title):
    return df[df.original_title == original_title]["index"].values[0]

### **BERT Embedding**

Once we get the combined data, we will use Sentence transformers for BERT Embedding. Install the library and download a pretrained model, we have used ‘bert-base-nli-mean-tokens’ pretrained model. You can provide any pre-trained model, here is the [list](https://www.sbert.net/docs/pretrained_models.html)

In [60]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.6 MB/s eta 0:00:00


In [61]:
from sentence_transformers import SentenceTransformer
bert = SentenceTransformer('bert-base-nli-mean-tokens')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Perform embedding of ‘combined_value’

In [62]:
#Get Embeddings
sentence_embeddings = bert.encode(df['combined_value'].tolist())

### **Cosine Similarity**

After calculating the encoding we will get our vectors, now we need to find similarities between those vectors. To do so we will use Cosine Similarity, which provides a similarity score between two vectors, where 0 indicates no similarity and 1 indicates complete similarity.

In [63]:
#Compute similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(sentence_embeddings)

Now when the user enters the movie name, the system compares the entered movie name score with other scores and recommends the movie to the user.

### **Testing**

In [64]:
movie_name = 'Toy Story'

In [65]:
distances = sorted(list(enumerate(similarity[index(movie_name)])), key = lambda x:x[1], reverse = True)

In [66]:
print(title(distances[1][0]), title(distances[2][0]), title(distances[3][0]), title(distances[4][0]), title(distances[5][0]), sep = "\n")

Toy Story 2
Toy Story 3
The Aristocats
The Boxtrolls
The Emoji Movie


In [67]:
def recommend(movie):
    distances = sorted(list(enumerate(similarity[index(movie_name)])), key = lambda x:x[1], reverse = True)
    for i in distances[1:6]:
        print(df.iloc[i[0]].original_title)

In [68]:
recommend('Toy Story')

Toy Story 2
Toy Story 3
The Aristocats
The Boxtrolls
The Emoji Movie


In [69]:
df['original_title'].values

array(['Toy Story', 'Jumanji', 'Grumpier Old Men', ...,
       'Shadow of the Blair Witch', 'The Burkittsville 7',
       'Siglo ng Pagluluwal'], dtype=object)

In [70]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')
import pickle

# Specify the path to the "abc" folder within your My Drive
folder_path = '/content/drive/MyDrive/recommender_system'

# Save the pickle file in the folder
pickle_path_1 = os.path.join(folder_path, 'mov_dict.pkl')
with open(pickle_path_1, 'wb') as f:
  pickle.dump(df.to_dict(), f)

pickle_path_2 = os.path.join(folder_path, 'similar.pkl')
with open(pickle_path_2, 'wb') as f:
  pickle.dump(similarity, f)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
